In [33]:
#%matplotlib inline
!pwd

/Users/kpadhikari/GitStuff/KPAdhikari/PythonStuff/WebScrapingStuff


Although I had visited this page (https://realpython.com/python-web-scraping-practical-introduction/#a-primer-on-regular-expressions) quite a while ago, and had thought of going through it step by step all the way but got distracted. Today, I am coming back and want to complete it. (2/19/2022)

# A Practical Introduction to Web Scraping in Python
by David Amos

Table of Contents

    Scrape and Parse Text From Websites
        Your First Web Scraper
        Extract Text From HTML With String Methods
        A Primer on Regular Expressions
        Extract Text From HTML With Regular Expressions
        Check Your Understanding
    Use an HTML Parser for Web Scraping in Python
        Install Beautiful Soup
        Create a BeautifulSoup Object
        Use a BeautifulSoup Object
        Check Your Understanding
    Interact With HTML Forms
        Install MechanicalSoup
        Create a Browser Object
        Submit a Form With MechanicalSoup
        Check Your Understanding
    Interact With Websites in Real Time
    Conclusion
    Additional Resources
    
Web scraping is the process of collecting and parsing raw data from the Web, and the Python community has come up with some pretty powerful web scraping tools.

The Internet hosts perhaps the greatest source of information—and misinformation—on the planet. Many disciplines, such as data science, business intelligence, and investigative reporting, can benefit enormously from collecting and analyzing data from websites.

In this tutorial, you’ll learn how to:

* Parse website data using string methods and regular expressions
* Parse website data using an HTML parser
* Interact with forms and other website components
    
**Note:** This tutorial is adapted from the chapter “Interacting With the Web” in Python Basics: A Practical Introduction to Python 3.

The book uses Python’s built-in IDLE editor to create and edit Python files and interact with the Python shell, so you will see occasional references to IDLE throughout this tutorial. However, you should have no problems running the example code from the editor and environment of your choice.

## Scrape and Parse Text From Websites
Collecting data from websites using an automated process is known as web scraping. Some websites explicitly forbid users from scraping their data with automated tools like the ones you’ll create in this tutorial. Websites do this for two possible reasons:

* The site has a good reason to protect its data. For instance, Google Maps doesn’t let you request too many results too quickly.
* Making many repeated requests to a website’s server may use up bandwidth, slowing down the website for other users and potentially overloading the server such that the website stops responding entirely.

**Important:** Before using your Python skills for web scraping, you should always check your target website’s acceptable use policy to see if accessing the website with automated tools is a violation of its terms of use. Legally, web scraping against the wishes of a website is very much a gray area.

Please be aware that the following techniques may be illegal when used on websites that prohibit web scraping.

Let’s start by grabbing all the HTML code from a single web page. You’ll use a page on Real Python that’s been set up for use with this tutorial.

## Your First Web Scraper
One useful package for web scraping that you can find in Python’s [standard library](https://docs.python.org/3/library/) is `urllib`, which contains tools for working with URLs. In particular, the [urllib.request](https://docs.python.org/3/library/urllib.request.html#module-urllib.request) module contains a function called `urlopen()` that can be used to open a URL within a program.

Let's open webpage at the url http://olympus.realpython.org/profiles/aphrodite:

In [12]:
from urllib.request import urlopen

# When I used the Following markdown-guide page, I got "HTTPError: HTTP Error 403: Forbidden"
#    which means - "the server has determined that you are not allowed access ..."
#url = "https://ia.net/writer/support/general/markdown-guide"
#url = "https://www.bbc.com/culture/article/20211215-why-the-tiny-house-movement-is-big"   #Works but too much stuff
url = "http://olympus.realpython.org/profiles/aphrodite"

page = urlopen(url)
print(type(page))
print(page)
page

<class 'http.client.HTTPResponse'>


Please remember that the urlopen() returns an HTTPResponse object as we see above.

To extract the html out of the 'page', first we use the HTTPResponse object's .read() method, which returns a sequence of bytes. Then we use .decode() to decode the bytes to a string using [UTF-8](https://realpython.com/python-encodings-guide/#unicode-vs-utf-8).

In [13]:
html_bytes = page.read()
html = html_bytes.decode('utf-8')

In [19]:
print(type(html), "\n")
print(len(html), html[0], "|", html[1], "|", html[6], "|", html[7], "\n\n\n")
print(html)

<class 'str'> 

278 < | h | 
 | < 



<html>
<head>
<title>Profile: Aphrodite</title>
</head>
<body bgcolor="yellow">
<center>
<br><br>
<img src="/static/aphrodite.gif" />
<h2>Name: Aphrodite</h2>
<br><br>
Favorite animal: Dove
<br><br>
Favorite color: Red
<br><br>
Hometown: Mount Olympus
</center>
</body>
</html>



Once you have the HTML as text, you can extract information from it in a couple of different ways.

## Extract Text From HTML With String Methods
One way to extract information from a web page’s HTML is to use [string methods](https://realpython.com/python-strings/#built-in-string-methods). For instance, you can use .find() to search through the text of the HTML for the `<title>` tags and extract the title of the web page.

Let’s extract the title of the web page you requested in the previous example. If you know the index of the first character of the title and the first character of the closing `</title>` tag, then you can use a [string slice](https://realpython.com/python-strings/#string-slicing) to extract the title.

Since .find() returns the index of the first occurrence of a substring, you can get the index of the opening `<title>` tag by passing the string "`<title>`" to `.find()`:

In [20]:
title_index = html.find("<title>")
title_index

14

In [21]:
for i in range(title_index+1):
    print(html[i])
    
#In the result of this loop, look at the gap at i=6 and i=13, they are created because at those 
#.   positions, we have the newline characters - something equivalent to "\n".

<
h
t
m
l
>


<
h
e
a
d
>


<


You don’t want the index of the `<title>` tag, though. You want the index of the title itself. To get the index of the first letter in the title, you can add the length of the string "`<title>`" to title_index:

In [23]:
print(len('<title>'))

7


In [26]:
start_index = title_index + len('<title>')
print(start_index)
print(html[start_index])

21
P


Now get the index of the closing </title> tag by passing the string "</title>" to .find():



In [28]:
end_index = html.find("</title>")
end_index

39

Finally, you can extract the title by slicing the html string:

In [29]:
title = html[start_index:end_index]
title

'Profile: Aphrodite'

Real-world HTML can be much more complicated and far less predictable than the HTML on the Aphrodite profile page. Here’s [another profile page](http://olympus.realpython.org/profiles/poseidon) with some messier HTML that you can scrape:

In [31]:
#Trying all together for <h1>
html[html.find("<h2>")+len("<h2>"):html.find("</h2>")]

'Name: Aphrodite'

Real-world HTML can be much more complicated and far less predictable than the HTML on the Aphrodite profile page. Here’s [another profile page](http://olympus.realpython.org/profiles/poseidon) with some messier HTML that you can scrape:

In [35]:
from urllib.request import urlopen
def getHTML(url):
    page = urlopen(url)
    return page.read().decode("utf-8")

html = getHTML("http://olympus.realpython.org/profiles/poseidon")

print(html)
html

<html>
<head>
<title >Profile: Poseidon</title>
</head>
<body bgcolor="yellow">
<center>
<br><br>
<img src="/static/poseidon.jpg" />
<h2>Name: Poseidon</h2>
<br><br>
Favorite animal: Dolphin
<br><br>
Favorite color: Blue
<br><br>
Hometown: Sea
</center>
</body>
</html>



'<html>\n<head>\n<title >Profile: Poseidon</title>\n</head>\n<body bgcolor="yellow">\n<center>\n<br><br>\n<img src="/static/poseidon.jpg" />\n<h2>Name: Poseidon</h2>\n<br><br>\nFavorite animal: Dolphin\n<br><br>\nFavorite color: Blue\n<br><br>\nHometown: Sea\n</center>\n</body>\n</html>\n'

In [37]:
def getFromTag(html, tagStr):
    properTag = "<" + tagStr + ">"
    start_index = html.find(properTag) + len(properTag)
    end_index = html.find("</" + tagStr + ">")
    return html[start_index:end_index]


'\n<head>\n<title >Profile: Poseidon'

In [38]:
getFromTag(html,"h2")

'Name: Poseidon'

In [39]:
getFromTag(html,"head")

'\n<title >Profile: Poseidon</title>\n'

In [40]:
getFromTag(html, "title")

'\n<head>\n<title >Profile: Poseidon'

Whoops! There’s a bit of HTML mixed in with the title. Why’s that?

The HTML for the `/profiles/poseidon` page looks similar to the `/profiles/aphrodite` page, but there’s a small difference. The opening `<title>` tag has an extra space before the closing angle bracket (`>`), rendering it as `<title >`.

`html.find("<title>")` returns -1 because the exact substring "`<title>`" doesn’t exist. When -1 is added to len("`<title>`"), which is 7, the start_index variable is assigned the value 6.

The character at index 6 of the string html is a newline character (\n) right before the opening angle bracket (<) of the `<head>` tag. This means that `html[start_index:end_index]` returns all the HTML starting with that newline and ending just before the `</title>` tag.

These sorts of problems can occur in countless unpredictable ways. You need a more reliable way to extract text from HTML.

## A Primer on Regular Expressions
**Regular expressions** — or **regexes** for short — <font color="magenta"><b>are patterns</b> that can be used to search for text within a string</font>. Python supports regular expressions through the standard library’s re module.

Note: Regular expressions aren’t particular to Python. They’re a general programming concept and can be used with any programming language.

To work with regular expressions, the first thing you need to do is <font color="magenta">import the re module</font>.

Regular expressions use special characters called **metacharacters** to denote different patterns. For instance, the <font color="magenta">asterisk character (*) stands for zero or more instances of whatever character comes/stands just before the asterisk</font>.

In the following example, you use findall() to find any text within a string that matches a given regular expression:

In [41]:
import re

re.findall("ab*c","ac")

['ac']

The first argument of re.findall() is the regular expression that you want to match, and the second argument is the string to test. In the above example, you search for the pattern "ab*c" in the string "ac".

The regular expression "ab*c" matches any part of the string that begins with an "a", ends with a "c", and has zero or more instances of "b" between the two<font color="magenta">(kp: and nothing else in between. For example, because of the 'd' in between a and c, it returns an empty list when the string is 'abdc' as we can see below)</font>. re.findall() returns a list of all matches. The string "ac" matches this pattern, so it’s returned in the [list](https://realpython.com/python-lists-tuples/).

Here’s the same pattern applied to different strings:

In [43]:
print(re.findall("ab*c","abcd"))
print(re.findall("ab*c","abcc"))
print(re.findall("ab*c","abcac"))
print(re.findall("ab*c","abdc"))

['abc']
['abc']
['abc', 'ac']
[]


Notice that if no match is found, then `findall()` returns an empty list.

Pattern matching is case sensitive. If you want to match this pattern regardless of the case, then you can pass a third argument with the value `re.IGNORECASE`:

In [45]:
print(re.findall("ab*c","AbcaC"))
print(re.findall("ab*c","AbcaC", re.IGNORECASE))

[]
['Abc', 'aC']


You can use a period (.) to stand for any single character <font color="red">(kp: but not empty or nothing, which is not a character)</font> in a regular expression. <font color="magenta">(kp: Looks to me that it is somewhat equivalent to the single character wild-card '?' in Linux/Unix shell/terminal commands, except it cannot be empty unlike with '?'. Remember, the asterisk '`*`' is the most generic wild card with anything possible in place of `*`)</font> For instance, you could find all the strings that contain the letters "a" and "c" separated by a single character as follows:

In [46]:
print(re.findall("a.c","abc"))
print(re.findall("a.c","abbc"))
print(re.findall("a.c","ac"))
print(re.findall("a.c","acc"))

['abc']
[]
[]
['acc']


The pattern `.*` inside a regular expression stands for any character repeated any number of times. For instance, `"a.*c"` can be used to find every substring that starts with "a" and ends with "c", regardless of which letter — or letters — are in between:

In [60]:
print(re.findall("a.*c","abc"))
print(re.findall("a.*c","abbc"))
print(re.findall("a.*c","ac"))
print(re.findall("a.*c","acc"))
print("######## kp: However, I find it puzzling that the following two don't give the exact same results")
print("      This is because Python’s regular expressions are greedy, meaning they try to find the longest ")
print("      possible match when characters like * are used, as is also seen with re.sub() example below.")
print("      So, we can use the non-greedy matching pattern *?, which works the same way as * except that")
print("      it matches the shortest possible string of text.")
print(re.findall("ab*c","abcac"))
print(re.findall("a.*c","abcac"))
print(re.findall("a.*?c","abcac"))

['abc']
['abbc']
['ac']
['acc']
######## kp: However, I find it puzzling that the following two don't give the exact same results
      This is because Python’s regular expressions are greedy, meaning they try to find the longest 
      possible match when characters like * are used, as is also seen with re.sub() example below.
      So, we can use the non-greedy matching pattern *?, which works the same way as * except that
      it matches the shortest possible string of text.
['abc', 'ac']
['abcac']
['abc', 'ac']


Often, you use `re.search()` to search for a particular pattern inside a string. This function is somewhat more complicated than `re.findall()` because it returns an object called a `MatchObject` that stores different groups of data. This is because there might be matches inside other matches, and `re.search()` returns every possible result.

The details of the MatchObject are irrelevant here. For now, just know that calling `.group()` on a MatchObject will return the first and most inclusive result, which in most cases is just what you want:

In [54]:
match_results = re.search("ab*c", "ABC", re.IGNORECASE)
match_results.group()

'ABC'

There’s one more function in the re module that’s useful for parsing out text. re.sub(), which is short for substitute, allows you to replace text in a string that matches a regular expression with new text. It behaves sort of like the .replace() string method.

The arguments passed to re.sub() are the regular expression, followed by the replacement text, followed by the string. Here’s an example:

In [55]:
string = "Everything is <replaced> if it's in <tags>."
string = re.sub("<.*>", "ELEPHANTS", string)
string

'Everything is ELEPHANTS.'

Perhaps that wasn’t quite what you expected to happen.

`re.sub()` uses the regular expression "`<.*>`" to find and replace everything between the first < and last >, which spans from the beginning of `<replaced>` to the end of `<tags>`. This is because <font color="red">Python’s regular expressions are <b>greedy</b></font>, meaning they try to find the longest possible match when characters like * are used. <font color="magenta">(kp: This explains why I saw a puzzling (initially) result when I tried `re.findall("a.*c","abcac")` above. Which was then "demystified" with an '?' added after `*` to the existing regex)</font>

Alternatively, you can use the non-greedy matching pattern `*?`, which works the same way as `*` except that it matches the shortest possible string of text:

In [61]:
string = "Everything is <replaced> if it's in <tags>."
string = re.sub("<.*?>", "ELEPHANTS", string)
string

"Everything is ELEPHANTS if it's in ELEPHANTS."

This time, `re.sub()` finds two matches, `<replaced>` and `<tags>`, and substitutes the string "ELEPHANTS" for both matches.

## Extract Text From HTML With Regular Expressions
Armed with all this knowledge, let’s now try to parse out the title from a [new profile page](http://olympus.realpython.org/profiles/dionysus), which includes this rather carelessly written line of HTML i.e. `<TITLE >Profile: Dionysus</title  / >`. The `.find()` method would have a difficult time dealing with the inconsistencies here, but with the clever use of regular expressions, you can handle this code quickly and efficiently:

In [66]:
html = getHTML("http://olympus.realpython.org/profiles/dionysus")
print(html)

<html>
<head>
<TITLE >Profile: Dionysus</title  / >
</head>
<body bgcolor="yellow">
<center>
<br><br>
<img src="/static/dionysus.jpg" />
<h2>Name: Dionysus</h2>
<img src="/static/grapes.png"><br><br>
Hometown: Mount Olympus
<br><br>
Favorite animal: Leopard <br>
<br>
Favorite Color: Wine
</center>
</body>
</html>



In [103]:
import re
from urllib.request import urlopen

def getFromTagRegEx(html, tagStr):
    #print(tagStr)
    pattern = "<" + tagStr + ".*?>.*?</" + tagStr + ".*?>"
    #print("RegEx pattern used: ", pattern)
    match_results = re.search(pattern, html, re.IGNORECASE)
    print(match_results)
    treasure = match_results.group()
    return re.sub("<.*?>", "", treasure) # Remove HTML tags    

In [104]:
getFromTag(html,"title")

'\n<head>\n<TITLE >Profile: Dionysus</title  / >\n</head>\n<body bgcolor="yellow">\n<center>\n<br><br>\n<img src="/static/dionysus.jpg" />\n<h2>Name: Dionysus</h2>\n<img src="/static/grapes.png"><br><br>\nHometown: Mount Olympus\n<br><br>\nFavorite animal: Leopard <br>\n<br>\nFavorite Color: Wine\n</center>\n</body>\n</html>'

In [105]:
getFromTagRegEx(html,"title")

<re.Match object; span=(14, 51), match='<TITLE >Profile: Dionysus</title  / >'>


'Profile: Dionysus'

Let’s take a closer look at the first regular expression in the pattern string by breaking it down into three parts:
<font color="magenta">(kp: The description below was based on the case when 'tagStr' was equal to "title" in the original tutorial. The function is actually my own. So, just think of the word 'title' anywhere 'tagStr' shows up in the function **getFromTagRegEx** above, when trying to appreciate the lines below.)</font>
* <title.*?> matches the opening `<TITLE >` tag in html. The `<title` part of the pattern matches with `<TITLE` because `re.search()` is called with `re.IGNORECASE`, and `.*?>` matches any text after `<TITLE` up to the first instance of `>`.
* `.*?` non-greedily matches all text after the opening `<TITLE >`, stopping at the first match for `</title.*?>`.
* `</title.*?>` differs from the first pattern only in its use of the / character, so it matches the closing `</title / >` tag in html.

The second regular expression, the string `"<.*?>"`, also uses the non-greedy .*? to match all the HTML tags in the title string. By replacing any matches with "", re.sub() removes all the tags and returns only the text.

**Note:** `Web scraping in Python or any other language can be tedious. No two websites are organized the same way, and HTML is often messy. Moreover, websites change over time. Web scrapers that work today are not guaranteed to work next year—or next week, for that matter!`

Regular expressions are a powerful tool when used correctly. This introduction barely scratches the surface. For more about regular expressions and how to use them, check out the two-part series [Regular Expressions: Regexes in Python](https://realpython.com/regex-python/).

### Check Your Understanding

Q: Write a program that grabs the full HTML from the following URL:  url = "http://olympus.realpython.org/profiles/dionysus "

Then use `.find()` to display the text following “Name:” and “Favorite Color:” (not including any leading spaces or trailing HTML tags that might appear on the same line).

In [91]:
html = getHTML("http://olympus.realpython.org/profiles/dionysus")
print(html)

<html>
<head>
<TITLE >Profile: Dionysus</title  / >
</head>
<body bgcolor="yellow">
<center>
<br><br>
<img src="/static/dionysus.jpg" />
<h2>Name: Dionysus</h2>
<img src="/static/grapes.png"><br><br>
Hometown: Mount Olympus
<br><br>
Favorite animal: Leopard <br>
<br>
Favorite Color: Wine
</center>
</body>
</html>



In [100]:
name = getFromTagRegEx(html, "h2")
#name = name + ">" #Just adding an arbitrary symbol ">" 
#name = re.search(name,"Name:.*?>", re.IGNORECASE)
#name = name.group()
print(name)

Name: Dionysus


In [102]:
pattern = "Name:" #"<" + tagStr + ".*?>.*?</" + tagStr + ".*?>"
re.findall(pattern, html, re.IGNORECASE)

['Name:']

In [112]:
pattern = "Name:.*?</"  #Pattern to find everything that starts with "Name:" and ends with "</"
match_results = re.search(pattern, html, re.IGNORECASE)
print(match_results)
treasure = match_results.group()
#https://note.nkmk.me/en/python-str-replace-translate-re-sub/
# Patterns delimited by | can be used to replace multiple strings with a single string
#re.sub("Name:", "", treasure) 
print(re.sub("Name:|</", "", treasure)) #It keeps the leading or trailing space
print(re.sub("Name:|</| ", "", treasure)) #It removes all the spaces (not a good option if we have 2 or more words)

<re.Match object; span=(141, 157), match='Name: Dionysus</'>
 Dionysus
Dionysus


### Trimming/Removing Leading and/or Trailing Whitespaces
https://www.regular-expressions.info/examples.html#:~:text=Search%20for%20%5E%5B%20%5Ct%5D,%5B%20%5Ct%5D%2B%24.

You can easily trim unnecessary whitespace from the start and the end of a string or the lines in a text file by doing a regex search-and-replace. Search for `^[ \t]+` and replace with nothing to delete leading whitespace (spaces and tabs). Search for `[ \t]+$` to trim trailing whitespace. Do both by combining the regular expressions into `^[ \t]+|[ \t]+$`. Instead of `[ \t]` which matches a space or a tab, you can expand the character class into `[ \t\r\n]` if you also want to strip line breaks. Or you can use the shorthand `\s` instead.


In [114]:
re.sub("Name:|</", "", treasure)

' Dionysus'

In [117]:
#https://stackoverflow.com/questions/7518172/regular-expression-python-remove-leading-whitespace
# "\s+" and "\s+$" are for leading and trailing white spaces.
re.sub("Name:|</|\s+|\s+$", "", treasure)

'Dionysus'

In [122]:
pattern = "Favorite Color:.*?\n"  #Pattern to find everything that starts with "Name:" and ends with "\n"
match_results = re.search(pattern, html, re.IGNORECASE)
print(match_results)
treasure = match_results.group()
re.sub("Favorite Color:|\n|\s+|\s+$", "", treasure)

<re.Match object; span=(268, 289), match='Favorite Color: Wine\n'>


'Wine'

## Solution to above exercise problem from the same tutorial site (revealed from 'Show/Hide' button)

First, import the urlopen function from the urlib.request module:

    from urllib.request import urlopen

Then open the URL and use the .read() method of the HTTPResponse object returned by urlopen() to read the page’s HTML:

In [123]:
url = "http://olympus.realpython.org/profiles/dionysus"
html_page = urlopen(url)
html_text = html_page.read().decode("utf-8")

.read() returns a byte string, so you use .decode() to decode the bytes using the UTF-8 encoding.

Now that you have the HTML source of the web page as a string assigned to the html_text variable, you can extract Dionysus’s name and favorite color from his profile. The structure of the HTML for Dionysus’s profile is the same as Aphrodite’s profile that you saw earlier.

You can get the name by finding the string "Name:" in the text and extracting everything that comes after the first occurence of the string and before the next HTML tag. That is, you need to extract everything after the colon (:) and before the first angle bracket (<). You can use the same technique to extract the favorite color.

The following [for loop](https://realpython.com/python-for-loop/) extracts this text for both the name and favorite color:

In [124]:
for string in ["Name: ", "Favorite Color:"]:
    string_start_idx = html_text.find(string)
    text_start_idx = string_start_idx + len(string)

    next_html_tag_offset = html_text[text_start_idx:].find("<")
    text_end_idx = text_start_idx + next_html_tag_offset

    raw_text = html_text[text_start_idx : text_end_idx]
    clean_text = raw_text.strip(" \r\n\t")
    print(clean_text)

Dionysus
Wine


It looks like there’s a lot going on in this forloop, but it’s just a little bit of arithmetic to calculate the right indices for extracting the desired text. Let’s break it down:

1. You use `html_text.find()` to find the starting index of the string, either "Name:" or "Favorite Color:", and then assign the index to `string_start_idx`.

2. Since the text to extract starts just after the colon in "Name:" or "Favorite Color:", you get the index of the the character immediately after the colon by adding the length of the string to `start_string_idx` and assign the result to `text_start_idx`.

3. You calculate the ending index of the text to extract by determining the index of the first angle bracket (<) relative to text_start_idx and assign this value to `next_html_tag_offset`. Then you add that value to `text_start_idx` and assign the result to `text_end_idx`.

4. You extract the text by `slicing html_text` from `text_start_idx` `to text_end_idx` and assign this string to `raw_text`.

5. You remove any whitespace from the beginning and end of `raw_text` using `.strip()` and assign the result to `clean_text`.

At the end of the loop, you use print() to display the extracted text. The final output looks like this:

    Dionysus
    Wine
    
This solution is one of many that solves this problem, so if you got the same output with a different solution, then you did great!

When you’re ready, you can move on to the next section.

## Use an HTML Parser for Web Scraping in Python
Although regular expressions are great for pattern matching in general, sometimes it’s easier to use an HTML parser that’s explicitly designed for parsing out HTML pages. There are many Python tools written for this purpose, but the [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/) library is a good one to start with.

### Install Beautiful Soup
To install Beautiful Soup, you can run the following in your terminal:

    `$ python3 -m pip install beautifulsoup4`
    
Run `pip show` to see the details of the package you just installed:

```
$ python3 -m pip show beautifulsoup4
Name: beautifulsoup4
Version: 4.9.1
Summary: Screen-scraping library
Home-page: http://www.crummy.com/software/BeautifulSoup/bs4/
Author: Leonard Richardson
Author-email: leonardr@segfault.org
License: MIT
Location: c:\realpython\venv\lib\site-packages
Requires:
Required-by:
In particular, notice that the latest version at the time of writing was 4.9.1.
```

<font color="magenta">I didn't have to do the installation because it seems the Anaconda already came pre-equipped with it.</font>

In [126]:
!which python

/Users/kpadhikari/opt/anaconda3/bin/python


In [138]:
!which beautifulsoup4  #Doesn't show anything because it's not an executable program like python, but just a library

## Create a BeautifulSoup Object
Type the following program into a new editor window:



In [134]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

html = getHTML("http://olympus.realpython.org/profiles/dionysus") #This func was defined above by me 

soup = BeautifulSoup(html, "html.parser")

In [136]:
print(type(html), type(soup), "\n")
print(soup)

<class 'str'> <class 'bs4.BeautifulSoup'> 

<html>
<head>
<title>Profile: Dionysus</title>
</head>
<body bgcolor="yellow">
<center>
<br/><br/>
<img src="/static/dionysus.jpg"/>
<h2>Name: Dionysus</h2>
<img src="/static/grapes.png"/><br/><br/>
Hometown: Mount Olympus
<br/><br/>
Favorite animal: Leopard <br/>
<br/>
Favorite Color: Wine
</center>
</body>
</html>



Above program does three things:

* Opens the URL http://olympus.realpython.org/profiles/dionysus using urlopen() from the urllib.request module

* Reads the HTML from the page as a string and assigns it to the html variable

* Creates a BeautifulSoup object and assigns it to the soup variable

The BeautifulSoup object assigned to soup is created with two arguments. The first argument is the HTML to be parsed, and the second argument, the string "html.parser", tells the object which parser to use behind the scenes. "html.parser" represents Python’s built-in HTML parser.

### Use a BeautifulSoup Object
Save and run the above program. When it’s finished running, you can use the soup variable in the interactive window to parse the content of html in various ways.

For example, BeautifulSoup objects have a .get_text() method that can be used to extract all the text from the document and automatically remove any HTML tags.

Type the following code into IDLE’s interactive window:

In [139]:
print(soup.get_text()) #It seems it removes all the html tags



Profile: Dionysus





Name: Dionysus

Hometown: Mount Olympus

Favorite animal: Leopard 

Favorite Color: Wine






There are a lot of blank lines in this output. These are the result of newline characters in the HTML document’s text. You can remove them with the string `.replace()` method if you need to.

Often, you need to get only specific text from an HTML document. <font color="red">Using Beautiful Soup first to extract the text and then using the `.find()` string method is sometimes easier than working with regular expressions.</font>

However, sometimes the HTML tags themselves are the elements that point out the data you want to retrieve. For instance, perhaps you want to retrieve the URLs for all the images on the page. These links are contained in the src attribute of `<img>` HTML tags.

In this case, you can use `find_all()` to return a list of all instances of that particular tag:

In [140]:
soup.find_all("img")

[<img src="/static/dionysus.jpg"/>, <img src="/static/grapes.png"/>]

This returns a list of all `<img>` tags in the HTML document. The objects in the list look like they might be strings representing the tags, but they’re actually instances of the Tag object provided by Beautiful Soup. Tag objects provide a simple interface for working with the information they contain.

Let’s explore this a little by first unpacking the Tag objects from the list:

In [142]:
image1, image2 = soup.find_all("img")

Each Tag object has a .name property that returns a string containing the HTML tag type:

In [143]:
image1.name

'img'

You can access the HTML attributes _(kp: such as color, font-size, style etc)_ of the Tag object by putting their name between square brackets, just as if the attributes were keys in a dictionary.

For example, the `<img src="/static/dionysus.jpg"/>` tag has a single attribute, src, with the value `"/static/dionysus.jpg"`. Likewise, an HTML tag such as the link `<a href="https://realpython.com" target="_blank">` has two attributes, href and target.

To get the source of the images in the Dionysus profile page, you access the src attribute using the dictionary notation mentioned above:

In [144]:
image1['src']

'/static/dionysus.jpg'

In [145]:
image2['src']

'/static/grapes.png'

Certain tags in HTML documents can be accessed by properties of the Tag object. For example, to get the `<title>` tag in a document, you can use the `.title` property <font color="magenta">kp: I think this works well only with those tags which has only one instance of it - such as 'title', 'heade' etc and not those such as 'p', 'br', 'hr' etc which can have more than one instances)</font>:

In [146]:
soup.title

<title>Profile: Dionysus</title>

If you look at the source of the Dionysus profile by navigating to the profile page, right-clicking on the page, and selecting View page source, then you’ll notice that the `<title>` tag as written in the document looks like this:

    <title >Profile: Dionysus</title/>
    
Beautiful Soup automatically cleans up the tags for you by removing the extra space in the opening tag and the extraneous forward slash (/) in the closing tag.

You can also retrieve just the string between the title tags with the .string property of the Tag object:



In [147]:
soup.title.string

'Profile: Dionysus'

In [151]:
soup.br #There are multiple instances of <br/> but it returns only one

<br/>

In [152]:
soup.head

<head>
<title>Profile: Dionysus</title>
</head>

One of the more useful features of Beautiful Soup is the ability to search for specific kinds of tags whose attributes match certain values. For example, if you want to find all the `<img>` tags that have a src attribute equal to the value `/static/dionysus.jpg`, then you can provide the following additional argument to .find_all():

In [153]:
soup.find_all('img', src='/static/dionysus.jpg')

[<img src="/static/dionysus.jpg"/>]

This example is somewhat arbitrary, and the usefulness of this technique may not be apparent from the example. If you spend some time browsing various websites and viewing their page sources, then you’ll notice that many websites have extremely complicated HTML structures.

When scraping data from websites with Python, you’re often interested in particular parts of the page. By spending some time looking through the HTML document, you can identify tags with unique attributes that you can use to extract the data you need.

Then, instead of relying on complicated regular expressions or using .find() to search through the document, you can directly access the particular tag you’re interested in and extract the data you need.

In some cases, you may find that Beautiful Soup doesn’t offer the functionality you need. The [lxml](http://lxml.de/) library is somewhat trickier to get started with but offers far more flexibility than Beautiful Soup for parsing HTML documents. You may want to check it out once you’re comfortable using Beautiful Soup.

    Note: HTML parsers like Beautiful Soup can save you a lot of time and effort when it comes to locating specific data in web pages. However, sometimes HTML is so poorly written and disorganized that even a sophisticated parser like Beautiful Soup can’t interpret the HTML tags properly.

    In this case, you’re often left with using .find() and regular expression techniques to try to parse out the information you need.

<font color="red">BeautifulSoup is great for scraping data from a website’s HTML, but it doesn’t provide any way to work with HTML forms. For example, if you need to search a website for some query and then scrape the results, then BeautifulSoup alone won’t get you very far.</font>

### Check your understanding
Write a program that grabs the full HTML from the [page](http://olympus.realpython.org/profiles) at the URL http://olympus.realpython.org/profiles.

Using Beautiful Soup, print out a list of all the links on the page by looking for HTML tags with the name `a` and retrieving the value taken on by the href attribute of each tag.

The final output should look like this:

    http://olympus.realpython.org/profiles/aphrodite
    http://olympus.realpython.org/profiles/poseidon
    http://olympus.realpython.org/profiles/dionysus
    


In [155]:
html = getHTML('http://olympus.realpython.org/profiles')
soup = BeautifulSoup(html, "html.parser")
soup

<html>
<head>
<title>All Profiles</title>
</head>
<body bgcolor="yellow">
<center>
<br/><br/>
<h1>All Profiles:</h1>
<br/><br/>
<h2>
<a href="/profiles/aphrodite">Aphrodite</a>
<br/><br/>
<a href="/profiles/poseidon">Poseidon</a>
<br/><br/>
<a href="/profiles/dionysus">Dionysus</a>
</h2>
</center>
</body>
</html>

In [158]:
soup.a

<a href="/profiles/aphrodite">Aphrodite</a>

In [157]:
soup.a['href']

'/profiles/aphrodite'

In [159]:
soup.h2

<h2>
<a href="/profiles/aphrodite">Aphrodite</a>
<br/><br/>
<a href="/profiles/poseidon">Poseidon</a>
<br/><br/>
<a href="/profiles/dionysus">Dionysus</a>
</h2>

In [160]:
soup.h2.a

<a href="/profiles/aphrodite">Aphrodite</a>

In [162]:
links = soup.find_all('a')
links

[<a href="/profiles/aphrodite">Aphrodite</a>,
 <a href="/profiles/poseidon">Poseidon</a>,
 <a href="/profiles/dionysus">Dionysus</a>]

In [164]:
for link in links:
    print(link['href'])

/profiles/aphrodite
/profiles/poseidon
/profiles/dionysus


In [165]:
for link in links:
    print('http://olympus.realpython.org/' + link['href'])

http://olympus.realpython.org//profiles/aphrodite
http://olympus.realpython.org//profiles/poseidon
http://olympus.realpython.org//profiles/dionysus


### The Solution to above exercise problem from Show/Hide 
First, import the urlopen function from the urlib.request module and the BeautifulSoup class from the bs4 package:

    from urllib.request import urlopen
    from bs4 import BeautifulSoup
Each link URL on the /profiles page is a relative URL, so create a base_url variable with the base URL of the website:

    base_url = "http://olympus.realpython.org"
    
You can build a full URL by concatenating base_url with a relative URL.

Now open the /profiles page with urlopen() and use .read() to get the HTML source:

    html_page = urlopen(base_url + "/profiles")
    html_text = html_page.read().decode("utf-8")
    
With the HTML source downloaded and decoded, you can create a new BeautifulSoup object to parse the HTML:

    soup = BeautifulSoup(html_text, "html.parser")
    
`soup.find_all("a")` returns a list of all links in the HTML source. You can loop over this list to print out all the links on the webpage:

    for link in soup.find_all("a"):
        link_url = base_url + link["href"]
        print(link_url)
    
The relative URL for each link can be accessed through the "href" subscript. Concatenate this value with `base_url` to create the full `link_url`.

When you’re ready, you can move on to the next section. <font color="magenta">(kp: Basically, that's what I did above myself without looking at this solution)</font>

## Interact With HTML Forms
The urllib module you’ve been working with so far in this tutorial is well suited for requesting the contents of a web page. <font color="red">Sometimes, though, you need to interact with a web page to obtain the content you need. For example, you might need to submit a form or click a button to display hidden content.</font>

Note: This tutorial is adapted from the chapter “Interacting With the Web” in [Python Basics: A Practical Introduction to Python 3](https://realpython.com/products/python-basics-book/). If you enjoy what you’re reading, then be sure to check out the rest of the book.

The Python standard library doesn’t provide a built-in means for working with web pages interactively, but many third-party packages are available from PyPI. Among these, [MechanicalSoup](https://github.com/hickford/MechanicalSoup) is a popular and relatively straightforward package to use.

<font color="red" size="5.0">In essence, MechanicalSoup installs what’s known as a headless browser, which is a web browser with no graphical user interface.</font> This browser is controlled programmatically via a Python program.

### Install MechanicalSoup
You can install MechanicalSoup with pip in your terminal:

    `$ python3 -m pip install MechanicalSoup`
    
You can now view some details about the package with pip show:
```
    $ python3 -m pip show mechanicalsoup
    Name: MechanicalSoup
    Version: 0.12.0
    Summary: A Python library for automating interaction with websites
    Home-page: https://mechanicalsoup.readthedocs.io/
    Author: UNKNOWN
    Author-email: UNKNOWN
    License: MIT
    Location: c:\realpython\venv\lib\site-packages
    Requires: requests, beautifulsoup4, six, lxml
    Required-by:
```    
In particular, notice that the latest version at the time of writing was `0.12.0`. You’ll need to close and restart your IDLE session for MechanicalSoup to load and be recognized after it’s been installed.

It seems the Anaconda doesn't have the 'mechanicalsoup' library because it gave `ModuleNotFoundError: No module named 'mechanicalsoup'`. So, I am installing it here.

In [167]:
!python3 -m pip install MechanicalSoup

  Using cached MechanicalSoup-1.1.0-py3-none-any.whl (19 kB)


### Create a Browser Object

Type the following into IDLE’s interactive window:

In [173]:
import mechanicalsoup
browser = mechanicalsoup.Browser()
print(type(browser))
print("\n", browser)

<class 'mechanicalsoup.browser.Browser'>



Browser objects represent the headless web browser. You can use them to request a page from the Internet by passing a URL to their .get() method:

    >>> url = "http://olympus.realpython.org/login"
    >>> page = browser.get(url)
    
page is a Response object that stores the response from requesting the URL from the browser:

    >>> page
    <Response [200]>
    
The number 200 represents the status code returned by the request. A status code of 200 means that the request was successful. An unsuccessful request might show a status code of 404 if the URL doesn’t exist or 500 if there’s a server error when making the request.

MechanicalSoup uses Beautiful Soup to parse the HTML from the request. page has a .soup attribute that represents a BeautifulSoup object:

    >>> type(page.soup)
    <class 'bs4.BeautifulSoup'>
    
You can view the HTML by inspecting the .soup attribute:



In [174]:
url = "http://olympus.realpython.org/login"
page = browser.get(url)
page

<Response [200]>

In [178]:
print(type(page))
print(type(page.soup), "\n -------kpa kpa kpa------")
print(page.soup)

<class 'requests.models.Response'>
<class 'bs4.BeautifulSoup'> 
 -------kpa kpa kpa------
<html>
<head>
<title>Log In</title>
</head>
<body bgcolor="yellow">
<center>
<br/><br/>
<h2>Please log in to access Mount Olympus:</h2>
<br/><br/>
<form action="/login" method="post" name="login">
Username: <input name="user" type="text"/><br/>
Password: <input name="pwd" type="password"/><br/><br/>
<input type="submit" value="Submit"/>
</form>
</center>
</body>
</html>



Notice this page has a `<form>` on it with `<input>` elements for a username and a password.

### Submit a Form With MechanicalSoup
Open the `/login` page (http://olympus.realpython.org/login) from the previous example in a browser and look at it yourself before moving on. Try typing in a random username and password combination. If you guess incorrectly, then the message “Wrong username or password!” is displayed at the bottom of the page.

However, if you provide the correct login credentials (username zeus and password ThunderDude), then you’re redirected to the /profiles page.

In the next example, you’ll see how to use MechanicalSoup to fill out and submit this form using Python!

The important section of HTML code is the login form—that is, everything inside the `<form>` tags. The `<form>` on this page has the name attribute set to login. This form contains two `<input>` elements, one named user and the other named pwd. The third `<input>` element is the Submit button.

Now that you know the underlying structure of the login form, as well as the credentials needed to log in, let’s take a look at a program. that fills the form out and submits it.

In a new editor window, type in the following program:



In [179]:
import mechanicalsoup

# 1
browser = mechanicalsoup.Browser()
url = "http://olympus.realpython.org/login"
login_page = browser.get(url)
login_html = login_page.soup

# 2
form = login_html.select("form")[0]
form.select("input")[0]["value"] = "zeus"
form.select("input")[1]["value"] = "ThunderDude"

# 3
profiles_page = browser.submit(form, login_page.url)

<Response [200]>


In [185]:
print(login_page)
print(login_html)
print(form)
print(profiles_page)
print(type(profiles_page))
print(profiles_page.url)
print(profiles_page.content)

<Response [200]>
<html>
<head>
<title>Log In</title>
</head>
<body bgcolor="yellow">
<center>
<br/><br/>
<h2>Please log in to access Mount Olympus:</h2>
<br/><br/>
<form action="/login" method="post" name="login">
Username: <input name="user" type="text" value="zeus"/><br/>
Password: <input name="pwd" type="password" value="ThunderDude"/><br/><br/>
<input type="submit" value="Submit"/>
</form>
</center>
</body>
</html>

<form action="/login" method="post" name="login">
Username: <input name="user" type="text" value="zeus"/><br/>
Password: <input name="pwd" type="password" value="ThunderDude"/><br/><br/>
<input type="submit" value="Submit"/>
</form>
<Response [200]>
<class 'requests.models.Response'>
http://olympus.realpython.org/profiles
b'<html>\n<head>\n<title>All Profiles</title>\n</head>\n<body bgcolor="yellow">\n<center>\n<br><br>\n<h1>All Profiles:</h1>\n<br><br>\n<h2>\n<a href="/profiles/aphrodite">Aphrodite</a>\n<br><br>\n<a href="/profiles/poseidon">Poseidon</a>\n<br><br>\n<a 

Let’s break down the above example:

1. You create a Browser instance and use it to request the URL http://olympus.realpython.org/login. You assign the HTML content of the page to the login_html variable using the .soup property.

2. `login_html.select("form")` returns a list of all `<form>` elements on the page. Since the page has only one `<form>` element, you can access the form by retrieving the element at index 0 of the list. The next two lines select the username and password inputs and set their value to "zeus" and "ThunderDude", respectively.

3. You submit the form with `browser.submit()`. Notice that you pass two arguments to this method, the form object and the URL of the login_page, which you access via `login_page.url`.

In the interactive window, you confirm that the submission successfully redirected to the /profiles page. If something had gone wrong, then the value of `profiles_page.url` would still be `"http://olympus.realpython.org/login"`.

<font color="Green">
    <b>Note:</b> Hackers can use automated programs like the one above to brute force logins by rapidly trying many different usernames and passwords until they find a working combination.

Besides this being highly illegal, almost all websites these days lock you out and report your IP address if they see you making too many failed requests, so don’t try it!
</font>

Now that we have the profiles_page variable set, let’s see how to programmatically obtain the URL for each link on the /profiles page.

To do this, you use .select() again, this time passing the string "a" to select all the `<a>` anchor elements on the page:

```
>>> links = profiles_page.soup.select("a")
Now you can iterate over each link and print the href attribute:
```

In [187]:
links = profiles_page.soup.select("a")
links

[<a href="/profiles/aphrodite">Aphrodite</a>,
 <a href="/profiles/poseidon">Poseidon</a>,
 <a href="/profiles/dionysus">Dionysus</a>]

In [189]:
for link in links:
    address = link['href'] #it gives the value of href attribute for the current 'a' or 'anchor' tag
    text = link.text       # it gives the hyperlinked text i.e. the text between the tags <a> and </a>
    print(f"{text}: {address}")

Aphrodite: /profiles/aphrodite
Poseidon: /profiles/poseidon
Dionysus: /profiles/dionysus


The URLs contained in each href attribute are relative URLs, which aren’t very helpful if you want to navigate to them later using MechanicalSoup. If you happen to know the full URL, then you can assign the portion needed to construct a full URL.

In this case, the base URL is just http://olympus.realpython.org. Then you can concatenate the base URL with the relative URLs found in the src attribute:

In [192]:
base_url = "http://olympus.realpython.org"

for link in links:
    address = base_url + link["href"]
    text = link.text
    print(f"{text}: {address}")

Aphrodite: http://olympus.realpython.org/profiles/aphrodite
Poseidon: http://olympus.realpython.org/profiles/poseidon
Dionysus: http://olympus.realpython.org/profiles/dionysus


You can do a lot with just .get(), .select(), and .submit(). That said, MechanicalSoup is capable of much more. To learn more about MechanicalSoup, check out the [official docs](https://mechanicalsoup.readthedocs.io/en/stable/).

### Check Your Understanding (From Show/Hide)
Use MechanicalSoup to provide the correct username (zeus) and password (ThunderDude) to the login form located at the URL http://olympus.realpython.org/login.

Once the form is submitted, display the title of the current page to determine that you’ve been redirected to the /profiles page.

Your program should print the text `<title>All Profiles</title>`.

#### My Solution

In [202]:
import mechanicalsoup

"""
# 1
browser = mechanicalsoup.Browser()
url = "http://olympus.realpython.org/login"
login_page = browser.get(url)
login_html = login_page.soup

# 2
form = login_html.select("form")[0]
form.select("input")[0]["value"] = "zeus"
form.select("input")[1]["value"] = "ThunderDude"

# 3
profiles_page = browser.submit(form, login_page.url)
"""
# I am simply reusing above lines (from previous cells)
print(profiles_page.soup.select("title"))
profiles_page.soup.select("title")

[<title>All Profiles</title>]


[<title>All Profiles</title>]

In [204]:
profiles_page.soup.select("title")[0] # To get rid of the brackets of the list, we must use the index of a particular cell
#Please note that 'select' returns a list of all the tags of the type entered as the argument such as 'title', 'a' etc.
#.   Sometimes, there could be only one instance such as with 'title' or 'head'. However, there could be more of 'h1', 'h2',
#.   'p', 'br', 'hr', etc. 

<title>All Profiles</title>

### Solution from the Tutorial (revealed by the Show/Hide button)
First, import the mechanicalsoup package and create a Broswer object:

```python
import mechanicalsoup

browser = mechanicalsoup.Browser()
```

Point the browser to the login page by passing the URL to browser.get() and grab the HTML with the .soup attribute:

```python
login_url = "http://olympus.realpython.org/login"
login_page = browser.get(login_url)
login_html = login_page.soup
```

login_html is a BeautifulSoup instance. Since the page has only a single form on it, you can access the form via login_html.form. Using .select(), select the username and password inputs and fill them with the username "zeus" and the password "ThunderDude":

```python
form = login_html.form
form.select("input")[0]["value"] = "zeus"
form.select("input")[1]["value"] = "ThunderDude"
```
Now that the form is filled out, you can submit it with browser.submit():

```python
profiles_page = browser.submit(form, login_page.url)
```

If you filled the form with the correct username and password, then profiles_page should actually point to the /profiles page. You can confirm this by printing the title of the page assigned to profiles_page:

    print(profiles_page.soup.title)

You should see the following text displayed:

`<title>All Profiles</title>`

If instead you see the text Log In or something else, then the form submission failed.

When you’re ready, you can move on to the next section.

In [205]:
print(profiles_page.soup.title)

<title>All Profiles</title>


## Interact With Websites in Real Time
Sometimes you want to be able to fetch real-time data from a website that offers continually updated information.

In the dark days before you learned Python programming, you had to sit in front of a browser, clicking the Refresh button to reload the page each time you wanted to check if updated content was available. But now you can automate this process using the .get() method of the MechanicalSoup Browser object.

Open your browser of choice and navigate to the URL http://olympus.realpython.org/dice. This /dice page simulates a roll of a six-sided die, updating the result each time you refresh the browser. Below, you’ll write a program that repeatedly scrapes the page for a new result.

The first thing you need to do is determine which element on the page contains the result of the die roll. Do this now by right-clicking anywhere on the page and selecting View page source. A little more than halfway down the HTML code is an `<h2>` tag that looks like this:

```html
<h2 id="result">4</h2>
```

The text of the `<h2>` tag might be different for you, but this is the page element you need for scraping the result.

    **Note:** For this example, you can easily check that there’s only one element on the page with id="result". Although the id attribute is supposed to be unique, in practice you should always check that the element you’re interested in is uniquely identified.

Let’s start by writing a simple program that opens the /dice page, scrapes the result, and prints it to the console:



In [211]:
import mechanicalsoup

browser = mechanicalsoup.Browser()
page = browser.get("http://olympus.realpython.org/dice")
tag = page.soup.select("#result")[0]
result = tag.text

print(f'tag = {tag} \n') #kp:

print(f"The result of your dice roll is: {result}")

tag = <h2 id="result">6</h2> 

The result of your dice roll is: 6


This example uses the BeautifulSoup object’s .select() method to find the element with id=result. The string "#result" that you pass to .select() uses the [CSS ID selector](https://developer.mozilla.org/en-US/docs/Web/CSS/ID_selectors) `#` to indicate that result is an id value.

To periodically get a new result, you’ll need to create a loop that loads the page at each step. So everything below the line `browser = mechanicalsoup.Browser()` in the above code needs to go in the body of the loop.

For this example, let’s get four rolls of the dice at ten-second intervals. To do that, the last line of your code needs to tell Python to pause running for ten seconds. You can do this with [sleep()](https://realpython.com/python-sleep/) from Python’s [time module](https://realpython.com/python-time-module/). sleep() takes a single argument that represents the amount of time to sleep in seconds.

Here’s an example that illustrates how sleep() works:

In [212]:
import time

print("I'm about to wait for five seconds...")
time.sleep(5)
print("Done waiting!")

I'm about to wait for five seconds...
Done waiting!


When you run this code, you’ll see that the "Done waiting!" message isn’t displayed until 5 seconds have passed from when the first print() function was executed.

For the die roll example, you’ll need to pass the number 10 to sleep(). Here’s the updated program:

In [216]:
import time
import mechanicalsoup

browser = mechanicalsoup.Browser()

for i in range(4):
    page = browser.get("http://olympus.realpython.org/dice")
    tag = page.soup.select("#result")[0]
    result = tag.text
    print(f"The result of your dice roll is: {result}")
    time.sleep(10)

The result of your dice roll is: 1
The result of your dice roll is: 4
The result of your dice roll is: 6
The result of your dice roll is: 5


When you run the program, you’ll immediately see the first result printed to the console. After ten seconds, the second result is displayed, then the third, and finally the fourth. What happens after the fourth result is printed?

The program continues running for another ten seconds before it finally stops!

Well, of course it does—that’s what you told it to do! But it’s kind of a waste of time. You can stop it from doing this by using an if statement to run time.sleep() for only the first three requests:

In [217]:
import time
import mechanicalsoup

browser = mechanicalsoup.Browser()

for i in range(4):
    page = browser.get("http://olympus.realpython.org/dice")
    tag = page.soup.select("#result")[0]
    result = tag.text
    print(f"The result of your dice roll is: {result}")

    # Wait 10 seconds if this isn't the last request
    if i < 3:
        time.sleep(10)

The result of your dice roll is: 4
The result of your dice roll is: 6
The result of your dice roll is: 1
The result of your dice roll is: 5


With techniques like this, you can scrape data from websites that periodically update their data. However, you should be aware that requesting a page multiple times in rapid succession can be seen as suspicious, or even malicious, use of a website.

<font color="green">
    <b>Important:</b> Most websites publish a Terms of Use document. You can often find a link to it in the website’s footer.

Always read this document before attempting to scrape data from a website. If you can’t find the Terms of Use, try to contact the website owner and ask them if they have any policies regarding request volume.

Failure to comply with the Terms of Use could result in your IP being blocked, so be careful and be respectful! </font>

It’s even possible to crash a server with an excessive number of requests, so you can imagine that many websites are concerned about the volume of requests to their server! Always check the Terms of Use and be respectful when sending multiple requests to a website.

## Conclusion
Although it’s possible to parse data from the Web using tools in Python’s standard library, there are many tools on PyPI that can help simplify the process.

In this tutorial, you learned how to:

* Request a web page using Python’s built-in urllib module
* Parse HTML using Beautiful Soup
* Interact with web forms using MechanicalSoup
* Repeatedly request data from a website to check for updates

Writing automated web scraping programs is fun, and the Internet has no shortage of content that can lead to all sorts of exciting projects.

Just remember, not everyone wants you pulling data from their web servers. Always check a website’s Terms of Use before you start scraping, and be respectful about how you time your web requests so that you don’t flood a server with traffic.

## Additional Resources
For more information on web scraping with Python, check out the following resources:

* [Beautiful Soup: Build a Web Scraper With Python](https://realpython.com/beautiful-soup-web-scraper-python/)
* [API Integration in Python](https://realpython.com/api-integration-in-python/)
* [Python & APIs: A Winning Combo for Reading Public Data](https://realpython.com/python-api/)

Note: If you enjoyed what you learned in this sample from Python Basics: A Practical Introduction to Python 3, then be sure to check out the rest of the book.